In [130]:
#Importando Bibliotecas
import pandas as pd
import numpy as np
import os
import datetime
from datetime import timedelta
from pathlib import Path

In [131]:
#Selecionando as Variáveis
ano_dt_base = 2020
tx_juros = 0.06
cres_sal = 0.01
cres_benef = 0.01

Colunas que vai precisar pro cálculo:

LOTAÇÃO	MATRICULA	GRUPO	CRITERIO DE APOSENTADORIA	SEXO	DTNASC	DTADM	DATACARGO	SALPART	CARREIRA	TSANT	CONJUGE	DTNASCONJ	DTNASDEPINV	DTNASDEPVAL

In [143]:
#Importando as bases
bd_ativos = pd.read_csv(str(Path(os.getcwd())) + '\\1.SERVIDORES.csv', encoding='UTF-8', sep=';')#, thousands=',')

#Convertendo os dados para float
bd_ativos['DT_NASC_SERVIDOR'] = pd.to_datetime(bd_ativos['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_ENTE'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_ING_CARGO'] = pd.to_datetime(bd_ativos['DT_ING_ENTE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['VL_BASE_CALCULO'] = bd_ativos['VL_BASE_CALCULO'].str.replace(',', '.').astype(float)
bd_ativos['DT_NASC_CONJUGE'] = pd.to_datetime(bd_ativos['DT_NASC_CONJUGE'], yearfirst=True, format='%d/%m/%Y')
bd_ativos['DT_NASC_NOVO'] = pd.to_datetime(bd_ativos['DT_NASC_NOVO'], yearfirst=True, format='%d/%m/%Y')

In [206]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = ano_dt_base - bd_ativos.DT_NASC_SERVIDOR.dt.strftime('%Y').astype(int) #Ano da Data-Base - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_ADM_ARRED'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)).astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
statis_ativos['IDADE_ADM_SEM_ARRED'] = (bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)
statis_ativos['IDADE_ADM_ARRED_CIMA'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) + 0.5).astype(int)
statis_ativos['IDADE_ADM_ARRED_BAIXO'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) - 0.5).astype(int) 
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos.DT_NASC_CONJUGE.mask(bd_ativos.DT_NASC_CONJUGE.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_CONJUGE = ano_dt_base - statis_ativos.IDADE_CONJUGE.dt.strftime('%Y').astype(int) #Calculando as Idades dos Cônjuges | Ano da Data-Base - Data de Nascimento do Cônjuge
statis_ativos.loc[(bd_ativos.CO_EST_CIVIL_SERVIDOR == 1) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 3) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 4) | (bd_ativos.CO_EST_CIVIL_SERVIDOR == 5) | (statis_ativos.IDADE_CONJUGE > 127), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_NOVO'] = bd_ativos.DT_NASC_NOVO.mask(bd_ativos.DT_NASC_NOVO.isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos.IDADE_NOVO = ano_dt_base - statis_ativos.IDADE_NOVO.dt.strftime('%Y').astype(int) #Calculando as Idades do Filho Mais Novo | Ano da Data-Base - Data de Nascimento do Filho Mais Novo
statis_ativos.loc[statis_ativos.IDADE_NOVO > 21, 'IDADE_NOVO'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
#statis_ativos['TP_PREFEITURA'] = round((2020-12-31 - bd_ativos.DT_ING_ENTE) / timedelta(days=365) - 0.5).astype(int) 
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIF_ETARIA'] = statis_ativos.IDADE_CONJUGE - statis_ativos.IDADE_SERVIDOR #Calculando a Diferença Etária do Cônjuge

#statistics_ativos['TSANT'] = bd_ativos['DT_ING_ENTE'] - pd.to_timedelta(bd_ativos['NU_TEMPO_RGPS'], unit='d')
statis_ativos.head(15)

,IDADE_SERVIDOR,IDADE_ADM_ARRED,IDADE_ADM_SEM_ARRED,IDADE_ADM_ARRED_CIMA,IDADE_ADM_ARRED_BAIXO,IDADE_CONJUGE,IDADE_NOVO,DIF_ETARIA
0,40,20,20.120548,21,20,NaN,16.0,NaN
1,45,43,42.501370,43,42,48.0,NaN,3.0
2,48,27,27.041096,28,27,NaN,18.0,NaN
3,31,23,23.304110,24,23,NaN,NaN,NaN
4,42,35,35.161644,36,35,NaN,NaN,NaN
5,51,29,29.468493,30,29,NaN,NaN,NaN
6,40,22,22.221918,23,22,53.0,NaN,13.0
7,40,34,34.315068,35,34,NaN,11.0,NaN
8,45,42,41.709589,42,41,NaN,NaN,NaN
9,41,30,30.273973,31,30,NaN,NaN,NaN


In [210]:
statis_ativos.DT_BASE.head()
bd_ativos.DT_NASC_SERVIDOR.head()
statis_ativos.DT_TESTE.sum()

16339

In [211]:
statis_ativos['DT_BASE'] = '31/12/2020'
statis_ativos.DT_BASE = pd.to_datetime(statis_ativos.DT_BASE, yearfirst=True, format='%d/%m/%Y')
statis_ativos['DT_TESTE'] = round((statis_ativos.DT_BASE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)).astype(int)
statis_ativos['DT_TESTE1'] = (statis_ativos.DT_BASE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)
statis_ativos['DT_TESTE2'] = round((statis_ativos.DT_BASE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) - 0.5).astype(int)
#statis_ativos.DT_TESTE = pd.to_datetime(statis_ativos.DT_TESTE)
statis_ativos.head(20)
#type(statis_ativos.DT_TESTE.loc[0])

,IDADE_SERVIDOR,IDADE_ADM_ARRED,IDADE_ADM_SEM_ARRED,IDADE_ADM_ARRED_CIMA,IDADE_ADM_ARRED_BAIXO,IDADE_CONJUGE,IDADE_NOVO,DIF_ETARIA,DT_BASE,DT_TESTE,DT_TESTE1,DT_TESTE2
0,40,20,20.120548,21,20,NaN,16.0,NaN,2020-12-31,41,40.638356,40
1,45,43,42.501370,43,42,48.0,NaN,3.0,2020-12-31,46,45.668493,45
2,48,27,27.041096,28,27,NaN,18.0,NaN,2020-12-31,49,48.805479,48
3,31,23,23.304110,24,23,NaN,NaN,NaN,2020-12-31,31,31.271233,31
4,42,35,35.161644,36,35,NaN,NaN,NaN,2020-12-31,43,42.898630,42
5,51,29,29.468493,30,29,NaN,NaN,NaN,2020-12-31,52,51.986301,51
6,40,22,22.221918,23,22,53.0,NaN,13.0,2020-12-31,40,40.484932,40
7,40,34,34.315068,35,34,NaN,11.0,NaN,2020-12-31,41,40.947945,40
8,45,42,41.709589,42,41,NaN,NaN,NaN,2020-12-31,46,45.668493,45
9,41,30,30.273973,31,30,NaN,NaN,NaN,2020-12-31,41,41.180822,41


In [10]:
#Calculando as Estatísticas dos Ativos
statis_ativos = pd.DataFrame()

statis_ativos['IDADE_SERVIDOR'] = ano_dt_base - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Ano da Data-Base - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
#statis_ativos['IDADE_ADM'] =  bd_ativos['DT_ING_ENTE'].dt.strftime('%Y').astype(int) - bd_ativos['DT_NASC_SERVIDOR'].dt.strftime('%Y').astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
statis_ativos['IDADE_ADM'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)).astype(int) #Data de Ingresso no Ente - Data de Nascimento do Servidor
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_CONJUGE'] = bd_ativos['DT_NASC_CONJUGE'].mask(bd_ativos['DT_NASC_CONJUGE'].isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos['IDADE_CONJUGE'] = ano_dt_base - statis_ativos['IDADE_CONJUGE'].dt.strftime('%Y').astype(int) #Calculando as Idades dos Cônjuges | Ano da Data-Base - Data de Nascimento do Cônjuge
statis_ativos.loc[(bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 1) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 3) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 4) | (bd_ativos['CO_EST_CIVIL_SERVIDOR'] == 5), 'IDADE_CONJUGE'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['IDADE_NOVO'] = bd_ativos['DT_NASC_NOVO'].mask(bd_ativos['DT_NASC_NOVO'].isnull(), '1800-01-01', axis=0)#, inplace=True) #Preenchendo as linhas vazias com alguma data
statis_ativos['IDADE_NOVO'] = ano_dt_base - statis_ativos['IDADE_NOVO'].dt.strftime('%Y').astype(int) #Calculando as Idades do Filho Mais Novo | Ano da Data-Base - Data de Nascimento do Filho Mais Novo
statis_ativos.loc[statis_ativos['IDADE_NOVO'] > 21, 'IDADE_NOVO'] = None #Removendo as datas inseridas
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
statis_ativos['DIF_ETARIA'] = statis_ativos['IDADE_CONJUGE'] - statis_ativos['IDADE_SERVIDOR'] #Calculando a Diferença Etária do Cônjuge

#statistics_ativos['TSANT'] = bd_ativos['DT_ING_ENTE'] - pd.to_timedelta(bd_ativos['NU_TEMPO_RGPS'], unit='d')
statis_ativos.head(15)

AttributeError: Can only use .dt accessor with datetimelike values

In [114]:
statis_ativos['IDADE_ADM_ARRED'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)).astype(int)
statis_ativos['IDADE_ADM_SEM_ARRED'] = (bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365)
statis_ativos['IDADE_ADM_ARRED_CIMA'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) + 0.5).astype(int)
statis_ativos['IDADE_ADM_ARRED_BAIXO'] = round((bd_ativos.DT_ING_ENTE - bd_ativos.DT_NASC_SERVIDOR) / timedelta(days=365) - 0.5).astype(int)
statis_ativos[['IDADE_ADM_ARRED', 'IDADE_ADM_SEM_ARRED', 'IDADE_ADM_ARRED_CIMA', 'IDADE_ADM_ARRED_BAIXO']]

,IDADE_ADM_ARRED,IDADE_ADM_SEM_ARRED,IDADE_ADM_ARRED_CIMA,IDADE_ADM_ARRED_BAIXO
0,20,20.120548,21,20
1,43,42.501370,43,42
2,27,27.041096,28,27
3,23,23.304110,24,23
4,35,35.161644,36,35
...,...,...,...,...
357,23,22.720548,23,22
358,24,24.361644,25,24
359,25,25.284932,26,25
360,30,30.260274,31,30


In [111]:
statis_ativos.IDADE_ADM3.sum()

11154